In [2]:
!pip install altair annoy bertopic umap-learn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 646 kB 6.1 MB/s 
     |████████████████████████████████| 76 kB 4.0 MB/s 
     |████████████████████████████████| 88 kB 4.9 MB/s 
     |████████████████████████████████| 5.2 MB 48.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 85 kB 4.5 MB/s 
     |████████████████████████████████| 1.1 MB 58.6 MB/s 
     |████████████████████████████████| 4.4 MB 37.3 MB/s 
     |████████████████████████████████| 1.2 MB 44.2 MB/s 
     |████████████████████████████████| 101 kB 8.3 MB/s 
     |████████████████████████████████| 636 kB 41.0 MB/s 
     |████████████████████████████████| 6.6 MB 49.2 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391585 sha256=3d2d8b4b95b6c0b2075e2feaa0f7f25d2488c94809ecf06bb

In [9]:
!wget https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_embeds.npy

--2022-07-21 01:59:19--  https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_embeds.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.13.240, 172.217.0.48, 172.217.2.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.13.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24576128 (23M) [application/octet-stream]
Saving to: ‘askhn3k_embeds.npy’

askhn3k_embeds.npy  100%[===================>]  23.44M   105MB/s    in 0.2s    

2022-07-21 01:59:19 (105 MB/s) - ‘askhn3k_embeds.npy’ saved [24576128/24576128]



In [3]:
#@title Import libraries (Run this cell to execute required code) {display-mode: "form"}
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
#from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
from sklearn.cluster import KMeans
from bertopic._ctfidf import ClassTFIDF
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import tensorflow as tf
import tensorflow_hub as hub

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [19]:
df = pd.read_csv('cleaned_data.csv',usecols=['url','search_text'])
df.head()

,url,search_text
0,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,workflows with jobs
1,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,workflows with jobs
2,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,monitoring
3,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,workflows
4,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,jobs


In [7]:
df.columns

Index(['url', 'search_text'], dtype='object')

In [8]:
all_intents = list(df['search_text'])

In [6]:
# # define the document embedding models to use for comparison
# module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
# model_use = hub.load(module_url)
# model_st1 = SentenceTransformer('all-mpnet-base-v2')
# model_st2 = SentenceTransformer('all-MiniLM-L6-v2')
# model_st3 = SentenceTransformer('paraphrase-mpnet-base-v2')



# def embed(model, model_type, sentences):
  
#     if model_type == 'use':
#             embeddings = model(sentences)
#     elif model_type == 'sentence transformer':
#             embeddings = model.encode(sentences)
#     return embeddings

# # generate embeddings for each model
# embeddings_use = embed(model_use, 'use', all_intents)
# embeddings_st1 = embed(model_st1, 'sentence transformer', all_intents)
# embeddings_st2 = embed(model_st2, 'sentence transformer', all_intents)
# embeddings_st3 = embed(model_st3, 'sentence transformer', all_intents)

# embeddings = ['embeddings_use','embeddings_st1','embeddings_st2','embeddings_st3']
# for embedding in embeddings:
#     print(embedding)

In [7]:
# all_embeddings = np.array(embeddings_st1)
# np.save('embeddings.npy', all_embeddings)

In [10]:
#Load the embeddings matrix
# embeds1 = np.load('askhn3k_embeds.npy')
embeds = np.load('embeddings.npy')
print(f'Loaded a DataFrame with {len(df)} rows and an embeddings matrix of dimensions {embeds.shape}')

Loaded a DataFrame with 3280 rows and an embeddings matrix of dimensions (3280, 768)


In [20]:
df.reset_index(inplace=True)

In [21]:
# Create the search index, pass the size of embedding
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('askhn.ann')

True

# 1- Given an existing post title, retrieve the most similar posts (nearest neighbor search using embeddings)
We can query neighbors of a specific post using get_nns_by_item.

In [22]:
# Choose an example (we'll retrieve others similar to it)
example_id = 45

# Retrieve nearest neighbors
similar_item_ids = search_index.get_nns_by_item(example_id,
                                                10, # Number of results to retrieve
                                                search_k = -1,include_distances=True)
# Format and print the text and distances
results = pd.DataFrame(data={'Query_post':df.iloc[example_id]['search_text'],'post_titles': df.iloc[similar_item_ids[0]]['search_text'],
                             'Distance': similar_item_ids[1]}).drop(example_id)

print(f"Query post:'{df.iloc[example_id]['search_text']}'\nNearest neighbors:")
results

Query post:'favorites and tags'
Nearest neighbors:


,Query_post,post_titles,Distance
1173,favorites and tags,cluster tags,0.996261
1181,favorites and tags,cluster tags,0.996261
1185,favorites and tags,cluster tags,0.996261
1187,favorites and tags,cluster tags,0.996261
2007,favorites and tags,cluster tags,0.996261
3199,favorites and tags,cluster tags,0.996261
522,favorites and tags,metastore,1.206370
542,favorites and tags,metastore,1.206370
893,favorites and tags,metastore,1.206370


In [12]:
def simalarity_embedding(num):
    data_list = []
    for ids in range(0,num):
        
        similar_item_ids = search_index.get_nns_by_item(ids,
                                                100, # Number of results to retrieve
                                                search_k = -1,include_distances=True)
        data = pd.DataFrame(data={'Query_post':df.iloc[ids]['search_text'],'post_titles': df.iloc[similar_item_ids[0]]['search_text'],
                             'Distance': similar_item_ids[1]}).drop(ids)
        # print(data)
        data_list.append(data)
    appended_data = pd.concat(data_list)
    return appended_data

In [13]:
data_k_nebighours = simalarity_embedding(embeds.shape[0])

In [14]:
embeds.shape[0]

3280

In [15]:
data_k_nebighours['Distance'] = data_k_nebighours['Distance'].round(3)

In [16]:
data_k_nebighours['Distance'].min()

0.0

In [17]:
df_drop_duplictes = data_k_nebighours.drop_duplicates(subset = ['Query_post', 'post_titles'],keep = 'last').reset_index(drop = True)

In [18]:
df_drop_duplictes.shape

(86915, 3)

In [19]:
df_in_threshold = df_drop_duplictes.loc[(df_drop_duplictes['Distance'] <= 0.8)]
df_outlier = df_drop_duplictes.loc[(df_drop_duplictes['Distance'] > 0.8)]

In [20]:
print('Data in threshhold',df_in_threshold.shape,'\nData outliers',df_outlier.shape)

Data in threshhold (3748, 3) 
Data outliers (83167, 3)


In [21]:
!altair --version

/bin/bash: altair: command not found


# 3- Plot the archive of articles by similarity
What if we want to browse the archive instead of only searching it. Let's plot out all the questions onto a 2D chart so you're able to visualize the posts in the archive and their similarities.

In [41]:
source = pd.DataFrame({
    'x': [1, 3, 5, 7, 9],
    'y': [1, 3, 5, 7, 9],
    'label': ['A', 'B', 'C', 'D', 'E']
})

points = alt.Chart(source).mark_point().encode(
    x='x:Q',
    y='y:Q'
)

text = points.mark_text(
    align='left',
    baseline='middle',
    dx=7
).encode(
    text='label'
)

points + text

alt.LayerChart(...)

In [47]:
reducer = umap.UMAP(n_neighbors=100) 
umap_embeds = reducer.fit_transform(embeds)

In [49]:
#https://stackoverflow.com/questions/57927065/altair-cant-access-filesystem-to-render-large-datasets-on-google-colab
!pip install altair_data_server
alt.data_transformers.enable('data_server')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


DataTransformerRegistry.enable('data_server')

In [50]:
df['x'] = umap_embeds[:,0]
df['y'] = umap_embeds[:,1]
# Plot
chart = alt.Chart(df).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    tooltip=[alt.Tooltip('search_text'),alt.Tooltip('url')]
).configure(background="#FDF7F0"
).properties(
    width=700,
    height=400,
    title='Ask HN: top 3,000 posts'
)

chart.interactive()


alt.Chart(...)

In [52]:
# Pick the number of clusters
n_clusters=8

# Cluster the embeddings
kmeans_model = KMeans(n_clusters=n_clusters, random_state=0)
classes = kmeans_model.fit_predict(umap_embeds)

In [53]:
# Extract the keywords for each cluster
documents =  df['search_text']
urls = df['url']
documents = pd.DataFrame({"Document": documents, "urls": urls,
                          "ID": range(len(documents)),
                          "Topic": None})
documents['Topic'] = classes
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
count_vectorizer = CountVectorizer(stop_words="english").fit(documents_per_topic.Document)
count = count_vectorizer.transform(documents_per_topic.Document)
words = count_vectorizer.get_feature_names()
ctfidf = ClassTFIDF().fit_transform(count).toarray()
words_per_class = {label: [words[index] for index in ctfidf[label].argsort()[-10:]] for label in documents_per_topic.Topic}
df['cluster'] = classes
df['keywords'] = df['cluster'].map(lambda topic_num: ", ".join(np.array(words_per_class[topic_num])[:]))
#Plot with clusters and keywords information

In [54]:
df.head()

,index,url,search_text,x,y,cluster,keywords
0,0,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,workflows with jobs,0.588027,0.424516,0,"time, storage, cli, firewall, job, library, ip, jobs, install, python"
1,1,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,workflows with jobs,0.621126,0.434982,0,"time, storage, cli, firewall, job, library, ip, jobs, install, python"
2,2,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,monitoring,0.076767,0.688631,0,"time, storage, cli, firewall, job, library, ip, jobs, install, python"
3,3,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,workflows,0.605824,0.726161,0,"time, storage, cli, firewall, job, library, ip, jobs, install, python"
4,4,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,jobs,0.964507,0.502054,0,"time, storage, cli, firewall, job, library, ip, jobs, install, python"


In [55]:
selection = alt.selection_multi(fields=['keywords'], bind='legend')

chart = alt.Chart(df).transform_calculate(
    url='https://news.ycombinator.com/item?id=' + alt.datum.id
).mark_circle(size=60, stroke='#666', strokeWidth=1, opacity=0.3).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    href='url:N',
    color=alt.Color('keywords:N', 
                    legend=alt.Legend(columns=1, symbolLimit=0, labelFontSize=14)
                   ),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    tooltip=['search_text', 'keywords', 'cluster']
).properties(
    width=800,
    height=500
).add_selection(
    selection
).configure_legend(labelLimit= 0).configure_view(
    strokeWidth=0
).configure(background="#FDF7F0").properties(
    title='Ask HN: Top 3,000 Posts'
)
chart.interactive()

alt.Chart(...)

In [29]:
df['keywords'].value_counts()

rest, jobs, control, token, ip, workspace, service, api, access, account             722
library, dataframe, parquet, apache, mlflow, file, metastore, python, data, spark    611
init, script, marketing, gitlab, storage, import, store, repos, feature, notebook    542
managed, clusters, region, sso, gcs, google, gke, vpc, cloud, cluster                461
connect, endpoint, azure, odbc, mount, dbutils, sql, jdbc, dbfs, databricks          354
column, view, sql, terraform, create, bigquery, tables, live, table, delta           335
power, catalog, bi, unity, public, preview, scope, secrets, quota, secret            137
reports, cluster, usage, delivery, catalog, unity, logs, log, scim, audit            118
Name: keywords, dtype: int64

In [30]:
df_in_threshold.head()

,Query_post,post_titles,Distance
0,workflows with jobs,workflows with jobs,0.000
1,workflows with jobs,workflows,0.728
55,databricks odbc and jdbc drivers,databricks jdbc,0.648
97,com.microsoft.sqlserver.jdbc.sqlserverexception: the connection is closed.,jdbc connect,0.800
218,west-4,'us-west4',0.762


In [31]:
temp1 = df.head()
temp2 = df_in_threshold.head()

In [32]:
df_join = df.merge(df_in_threshold, left_on= 'search_text', right_on='Query_post', how='inner')[['url','cluster','search_text','keywords','post_titles','Distance']]

In [33]:
df_join

,url,cluster,search_text,keywords,post_titles,Distance
0,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,1,workflows with jobs,"rest, jobs, control, token, ip, workspace, service, api, access, account",workflows with jobs,0.000
1,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,1,workflows with jobs,"rest, jobs, control, token, ip, workspace, service, api, access, account",workflows,0.728
2,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,1,workflows with jobs,"rest, jobs, control, token, ip, workspace, service, api, access, account",workflows with jobs,0.000
3,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,1,workflows with jobs,"rest, jobs, control, token, ip, workspace, service, api, access, account",workflows,0.728
4,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,1,workflows with jobs,"rest, jobs, control, token, ip, workspace, service, api, access, account",workflows with jobs,0.000
...,...,...,...,...,...,...
9988,https://docs.gcp.databricks.com/getting-started/spark/dataframes.html,0,pyspark dataframe from gcp bucket,"library, dataframe, parquet, apache, mlflow, file, metastore, python, data, spark",pyspark api,0.786
9989,https://docs.microsoft.com/en-us/azure/databricks/spark/latest/spark-sql/language-manual/functions/find_in_set,3,how to find the azure databricks dbu usage,"connect, endpoint, azure, odbc, mount, dbutils, sql, jdbc, dbfs, databricks",azure databricks sql,0.743
9990,https://docs.microsoft.com/en-us/azure/databricks/spark/latest/spark-sql/language-manual/functions/find_in_set,3,how to find the azure databricks dbu usage,"connect, endpoint, azure, odbc, mount, dbutils, sql, jdbc, dbfs, databricks",azure databricks api,0.788
9991,https://docs.microsoft.com/en-us/azure/databricks/spark/latest/spark-sql/language-manual/functions/xpath_string,0,typeerror 'str' object is not callable pyspark,"library, dataframe, parquet, apache, mlflow, file, metastore, python, data, spark",pyspark.pandas,0.729


In [34]:
df_join = df_join.drop_duplicates(subset = ['search_text', 'post_titles'],keep = 'last').reset_index(drop = True)

In [35]:
df_join.shape

(3748, 6)

In [36]:
df_join.head(10)

,url,cluster,search_text,keywords,post_titles,Distance
0,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,1,workflows with jobs,"rest, jobs, control, token, ip, workspace, service, api, access, account",workflows with jobs,0.000
1,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,1,workflows with jobs,"rest, jobs, control, token, ip, workspace, service, api, access, account",workflows,0.728
2,https://docs.gcp.databricks.com/data-governance/index.html,1,monitoring,"rest, jobs, control, token, ip, workspace, service, api, access, account",monitoring,0.000
3,https://docs.gcp.databricks.com/data-governance/index.html,1,monitoring,"rest, jobs, control, token, ip, workspace, service, api, access, account",model monitoring,0.793
4,https://docs.microsoft.com/en-us/azure/databricks/data-engineering/,1,workflows,"rest, jobs, control, token, ip, workspace, service, api, access, account",workflows,0.000
5,https://docs.microsoft.com/en-us/azure/databricks/data-engineering/,1,workflows,"rest, jobs, control, token, ip, workspace, service, api, access, account",workflow,0.412
6,https://docs.microsoft.com/en-us/azure/databricks/data-engineering/,1,workflows,"rest, jobs, control, token, ip, workspace, service, api, access, account",workflows with jobs,0.728
7,https://docs.gcp.databricks.com/data-engineering/jobs/jobs.html,1,jobs,"rest, jobs, control, token, ip, workspace, service, api, access, account",jobs,0.000
8,https://docs.gcp.databricks.com/data-engineering/jobs/jobs.html,1,jobs,"rest, jobs, control, token, ip, workspace, service, api, access, account",job,0.599
9,https://docs.microsoft.com/en-us/azure/databricks/data-engineering/jobs/jobs,1,job,"rest, jobs, control, token, ip, workspace, service, api, access, account",job,0.000


In [37]:
df_join['cluster_url'] =  df_join[['cluster', 'url']].values.tolist()

In [38]:
df_join

,url,cluster,search_text,keywords,post_titles,Distance,cluster_url
0,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,1,workflows with jobs,"rest, jobs, control, token, ip, workspace, service, api, access, account",workflows with jobs,0.000,"[1, https://docs.gcp.databricks.com/data-engineering/jobs/index.html]"
1,https://docs.gcp.databricks.com/data-engineering/jobs/index.html,1,workflows with jobs,"rest, jobs, control, token, ip, workspace, service, api, access, account",workflows,0.728,"[1, https://docs.gcp.databricks.com/data-engineering/jobs/index.html]"
2,https://docs.gcp.databricks.com/data-governance/index.html,1,monitoring,"rest, jobs, control, token, ip, workspace, service, api, access, account",monitoring,0.000,"[1, https://docs.gcp.databricks.com/data-governance/index.html]"
3,https://docs.gcp.databricks.com/data-governance/index.html,1,monitoring,"rest, jobs, control, token, ip, workspace, service, api, access, account",model monitoring,0.793,"[1, https://docs.gcp.databricks.com/data-governance/index.html]"
4,https://docs.microsoft.com/en-us/azure/databricks/data-engineering/,1,workflows,"rest, jobs, control, token, ip, workspace, service, api, access, account",workflows,0.000,"[1, https://docs.microsoft.com/en-us/azure/databricks/data-engineering/]"
...,...,...,...,...,...,...,...
3743,https://docs.gcp.databricks.com/getting-started/spark/dataframes.html,0,pyspark dataframe from gcp bucket,"library, dataframe, parquet, apache, mlflow, file, metastore, python, data, spark",pyspark api,0.786,"[0, https://docs.gcp.databricks.com/getting-started/spark/dataframes.html]"
3744,https://docs.microsoft.com/en-us/azure/databricks/spark/latest/spark-sql/language-manual/functions/find_in_set,3,how to find the azure databricks dbu usage,"connect, endpoint, azure, odbc, mount, dbutils, sql, jdbc, dbfs, databricks",azure databricks sql,0.743,"[3, https://docs.microsoft.com/en-us/azure/databricks/spark/latest/spark-sql/language-manual/functions/find_in_set]"
3745,https://docs.microsoft.com/en-us/azure/databricks/spark/latest/spark-sql/language-manual/functions/find_in_set,3,how to find the azure databricks dbu usage,"connect, endpoint, azure, odbc, mount, dbutils, sql, jdbc, dbfs, databricks",azure databricks api,0.788,"[3, https://docs.microsoft.com/en-us/azure/databricks/spark/latest/spark-sql/language-manual/functions/find_in_set]"
3746,https://docs.microsoft.com/en-us/azure/databricks/spark/latest/spark-sql/language-manual/functions/xpath_string,0,typeerror 'str' object is not callable pyspark,"library, dataframe, parquet, apache, mlflow, file, metastore, python, data, spark",pyspark.pandas,0.729,"[0, https://docs.microsoft.com/en-us/azure/databricks/spark/latest/spark-sql/language-manual/functions/xpath_string]"


In [39]:
df_join.to_csv('cluster_search_text_mapping.csv')

In [40]:
df_group = df_join.groupby(['search_text','Distance'])

In [41]:
df_group.first().head(5)

,,url,cluster,keywords,post_titles,cluster_url
search_text,Distance,,,,,
"""compute engine""",0.000,https://docs.gcp.databricks.com/administration-guide/cloud-configurations/gcp/firewall.html,0,"library, dataframe, parquet, apache, mlflow, file, metastore, python, data, spark","""compute engine""","[0, https://docs.gcp.databricks.com/administration-guide/cloud-configurations/gcp/firewall.html]"
"""data profile""",0.537,https://docs.microsoft.com/en-us/azure/databricks/release-notes/product/2021/november,0,"library, dataframe, parquet, apache, mlflow, file, metastore, python, data, spark",data profiles,"[0, https://docs.microsoft.com/en-us/azure/databricks/release-notes/product/2021/november]"
"""dbfs_fuse_version=1""",0.760,https://docs.microsoft.com/en-us/azure/databricks/release-notes/runtime/7.4,3,"connect, endpoint, azure, odbc, mount, dbutils, sql, jdbc, dbfs, databricks",configure dbfs,"[3, https://docs.microsoft.com/en-us/azure/databricks/release-notes/runtime/7.4]"
"""failed to establish a new connection: [errno 110] connection timed out""",0.779,https://docs.microsoft.com/en-us/azure/databricks/kb/python/import-custom-ca-cert,5,"managed, clusters, region, sso, gcs, google, gke, vpc, cloud, cluster","a connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond","[5, https://docs.microsoft.com/en-us/azure/databricks/kb/python/import-custom-ca-cert]"
"""invalid access token""",0.270,https://kb.databricks.com/en_US/dev-tools/use-tcpdump-create-pcap-files,1,"rest, jobs, control, token, ip, workspace, service, api, access, account",invalid access token,"[1, https://kb.databricks.com/en_US/dev-tools/use-tcpdump-create-pcap-files]"


In [42]:
df_group.first().to_excel('cluster_search_text_mapping.xlsx')

In [43]:
len(df_join['search_text'].unique())

1111

In [44]:
df.shape

(3280, 7)

In [45]:
len(df['search_text'].unique())

1760